# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import nltk
from nltk.tokenize import word_tokenize,wordpunct_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.ensemble import BaggingRegressor as bgr

from sklearn.svm import SVC
import unittest
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV

In [3]:
# load data from database
engine = sqlalchemy.create_engine('sqlite:///InsertDatabaseName.db')
df = pd.read_sql_table('InsertTableName', engine)

X = df['message']
y = df.drop(['id','message','original','genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
# init Lemmatizer and Tokenizer for acceptable NLP words (no punctuation)
wnl = WordNetLemmatizer()
tokenizer = nltk.RegexpTokenizer(r"\w+")

# tokenize the text
def tokenize(text):
    return tokenizer.tokenize(text.lower())

# clean the tokenized text
def clean_tokenized_text(text):
    text = tokenize(text)
    text = [wnl.lemmatize(word) for word in text if word not in stopwords.words('english')]
    return text

In [5]:
'''
Unit test for clean_tokenized_text
'''
class Unit_Test(unittest.TestCase):
        
    def test_clean_tokenized_text(self):
        text = 'Am I Real or not?'
        self.assertEqual(clean_tokenized_text(text), ['real'])

unittest.main(argv=[''], verbosity=2, exit=False)


test_clean_tokenized_text (__main__.Unit_Test) ... /opt/conda/lib/python3.6/site-packages/nltk/corpus/reader/wordlist.py:28: ResourceWarning: unclosed file <_io.BufferedReader name='/root/nltk_data/corpora/stopwords/english'>
  return concat([self.open(f).read() for f in fileids])
ok

----------------------------------------------------------------------
Ran 1 test in 2.240s

OK


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=clean_tokenized_text)),
                     ('tfidf', TfidfTransformer()),
                     ('rf', MultiOutputClassifier(rf()))
                    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [8]:
import time

s = time.time()

pipeline.fit(X_train, y_train)

print('time to fit: {} min'.format(round((time.time()-s)/60),1))

time to fit: 2 min


### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
s = time.time()
y_pred = pipeline.predict(X_test)

print('time to predict: {} min'.format(round((time.time()-s)/60),1))

time to predict: 0 min


In [10]:
'''
Iterate through the classes and compute f1, precision and accuracy
'''

for i,cat in enumerate(y_test.columns.tolist()):
    print('For category "{}" this is the results:'.format(cat))
    print(classification_report(y_pred=y_pred.reshape(36,-1)[i].astype('int'),
                                y_true=np.array(y_test).reshape(36,-1)[i].astype('int'),

                               )
         )
    print('--------------------------')
     
          


For category "related" this is the results:
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5890
          1       0.85      0.49      0.62       617

avg / total       0.94      0.94      0.94      6507

--------------------------
For category "request" this is the results:
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5946
          1       0.79      0.47      0.59       561

avg / total       0.94      0.94      0.94      6507

--------------------------
For category "offer" this is the results:
             precision    recall  f1-score   support

          0       0.95      0.99      0.97      5899
          1       0.81      0.44      0.57       608

avg / total       0.93      0.94      0.93      6507

--------------------------
For category "aid_related" this is the results:
             precision    recall  f1-score   support

          0       0.96      0.99      0.97

### 6. Improve your model
Use grid search to find better parameters. 

In [27]:
parameters = {#'rf__estimator__bootstrap': [True, False],
             'rf__estimator__n_estimators': [200, 600, 2000]}
             #,'rf__estimator__max_features': ['auto', 'sqrt']}

cv = GridSearchCV(pipeline, parameters)

In [25]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'vect', 'tfidf', 'rf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'rf__estimator__bootstrap', 'rf__estimator__class_weight', 'rf__estimator__criterion', 'rf__estimator__max_depth', 'rf__estimator__max_features', 'rf__estimator__max_leaf_nodes', 'rf__estimator__min_impurity_decrease', 'rf__estimator__min_impurity_split', 'rf__estimator__min_samples_leaf', 'rf__estimator__min_samples_split', 'rf__estimator__min_weight_fraction_leaf', 'rf__estimator__n_estimators', 'rf__estimator__n_jobs', 'rf__estimator__oob_score', 'rf__estimator__random_state', 'rf__estimator__verbose', 'rf__estimator__warm_start', 'rf__

In [33]:
import time

s = time.time()

cv.fit(X_train, y_train)

print('time to fit: {} min'.format(round((time.time()-s)/60),1))

KeyboardInterrupt: 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
s = time.time()
y_pred = cv.predict(X_test)

print('time to predict: {} min'.format(round((time.time()-s)/60),1))

In [ ]:
'''
Iterate through the classes and compute f1, precision and accuracy
'''

for i,cat in enumerate(y_test.columns.tolist()):
    print('For category "{}" this is the results:'.format(cat))
    print(classification_report(y_pred=y_pred.reshape(36,-1)[i].astype('int'),
                                y_true=np.array(y_test).reshape(36,-1)[i].astype('int'),

                               )
         )
    print('--------------------------')
     
          


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
pipeline_bg = Pipeline([('vect', CountVectorizer(tokenizer=clean_tokenized_text)),
                     ('tfidf', TfidfTransformer()),
                     ('bg', MultiOutputClassifier(bgr()))
                    ])

In [ ]:
import time

s = time.time()

pipeline_bg.fit(X_train, y_train)

print('time to fit: {} min'.format(round((time.time()-s)/60),1))

In [ ]:
s = time.time()
y_pred_bgr = pipeline_bg.predict(X_test)

print('time to predict: {} min'.format(round((time.time()-s)/60),1))

In [ ]:
'''
Iterate through the classes and compute f1, precision and accuracy
'''

for i,cat in enumerate(y_test.columns.tolist()):
    print('For category "{}" this is the results:'.format(cat))
    print(classification_report(y_pred=y_pred_bgr.reshape(36,-1)[i].astype('int'),
                                y_true=np.array(y_test).reshape(36,-1)[i].astype('int'),

                               )
         )
    print('--------------------------')
     
          


### 9. Export your model as a pickle file

In [30]:
# loading library
import pickle

In [32]:
# create an iterator object with write permission - model.pkl
with open('model_pkl', 'wb') as files:
    pickle.dump(pipeline, files)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [ ]:
import sys
# import libraries
import pandas as pd
import numpy as np
import sqlalchemy
import nltk
from nltk.tokenize import word_tokenize,wordpunct_tokenize
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier as rf

from sklearn.svm import SVC
import unittest
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score,precision_score,recall_score,accuracy_score

from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle


def load_data(database_filepath):
    # load data from database
    engine = sqlalchemy.create_engine('sqlite:///InsertDatabaseName.db')
    df = pd.read_sql_table('InsertTableName', engine)

    X = df['message']
    y = df.drop(['id','message','original','genre'], axis=1)
    category_names = y.columns.tolist()
    
    return X, y, category_names


def tokenize(text):  
    # init Lemmatizer and Tokenizer for acceptable NLP words (no punctuation)
    wnl = WordNetLemmatizer()
    tokenizer = nltk.RegexpTokenizer(r"\w+")   

    text = tokenizer.tokenize(text.lower())
    text = [wnl.lemmatize(word) for word in text if word not in stopwords.words('english')]
    return text



def build_model():
    pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('rf', MultiOutputClassifier(rf()))
                    ])
    
    return pipeline


def evaluate_model(model, X_test, Y_test, category_names):
    y_pred = model.predict(X_test)

    '''
    Iterate through the classes and compute f1, precision and accuracy
    '''

    for i,cat in enumerate(category_names):
        print('For category "{}" this is the results:'.format(cat))
        print(classification_report(y_pred=y_pred.reshape(36,-1)[i].astype('int'),
                                    y_true=np.array(Y_test).reshape(36,-1)[i].astype('int'),

                                   )
             )
        print('--------------------------')





def save_model(model, model_filepath):
    # create an iterator object with write permission - model.pkl
    with open('model_pkl', 'wb') as files:
        pickle.dump(model, files)


def main():
    if len(sys.argv) == 3:
        database_filepath, model_filepath = sys.argv[1:]
        print('Loading data...\n    DATABASE: {}'.format(database_filepath))
        X, Y, category_names = load_data(database_filepath)
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
        
        print('Building model...')
        model = build_model()
        
        print('Training model...')
        model.fit(X_train, Y_train)
        
        print('Evaluating model...')
        evaluate_model(model, X_test, Y_test, category_names)

        print('Saving model...\n    MODEL: {}'.format(model_filepath))
        save_model(model, model_filepath)

        print('Trained model saved!')

    else:
        print('Please provide the filepath of the disaster messages database '\
              'as the first argument and the filepath of the pickle file to '\
              'save the model to as the second argument. \n\nExample: python '\
              'train_classifier.py ../data/DisasterResponse.db classifier.pkl')


if __name__ == '__main__':
    main()